# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from tools import *

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) 
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

---
### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
---
#### For the query above, the data will be filtered based on sessionId and itemInsession. Therefore the first table should be indexed based on this two columns. First table "music_library" will be created below based on this criteria.
---


In [8]:
create_table_query = "CREATE TABLE IF NOT EXISTS music_library "
create_table_query = create_table_query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

---
#### Now the data should be inserted into the database. For this task, the defined insertInto function will be used.
---

In [9]:
insertInto(session, 'event_datafile_new.csv', 'music_library', ('sessionId', 'itemInSession', 'artist', 'song', 'length'))

Data inserting into music_library completed.


---
- [x] Query Design
- [x] Table Creation
- [ ] Query Test
#### Query is ready to be run!
---

In [10]:
query1 = "SELECT artist, song, length FROM music_library where sessionId=338 AND itemInSession=4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print(row)    

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


---
### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
---
#### For the query above we have multiple criteria.
##### - Two where clause items (userid and sessionid)
##### - sort by item (itemInSession) - WITH CLUSTERING ORDER BY
#### Therefore the primary key should include "userid", "sessionid", "itemInSession". It will be PRIMARY KEY ((userId, sessionId), itemInSession))
---

In [11]:
create_table_query = "CREATE TABLE IF NOT EXISTS user_table "
create_table_query = create_table_query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession DESC)" 
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

---
##### The table has been created. Again, the "insertInto" function will be used to push the data.
---


In [12]:
insertInto(session, 'event_datafile_new.csv', 'user_table', ('userId', 'sessionId', 'itemInSession', 'artist', 'song', 'firstName', 'lastName'))

Data inserting into user_table completed.


#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
query2 = "SELECT artist, song, firstName, lastName, itemInSession FROM user_table where userId=10 AND sessionId=182"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
for row in rows:
    print(row)

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz', iteminsession=3)
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz', iteminsession=2)
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz', iteminsession=1)
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz', iteminsession=0)


---
### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
---
#### For the query above, the data will be filtered based on song. However this is not enough to identify each row uniquely. Therefore song column will be paired with userId since user info is requested.
---

In [14]:
#Updated after feedback
# 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_table_query = "CREATE TABLE IF NOT EXISTS user_log "
create_table_query = create_table_query + "(userId int, song text, sessionId int, itemInSession int, artist text, firstName text, lastName text, PRIMARY KEY (userId, song)) " 
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

---
##### The table has been created. Again, the "insertInto" function will be used to push the data.
---

In [15]:
#Updated after feedback
insertInto(session, 'event_datafile_new.csv', 'user_log', ('userId', 'song', 'firstName', 'lastName'))

Data inserting into user_log completed.


---
##### Now query is ready to go!
---

In [16]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "SELECT  firstName, lastName  FROM user_log where song='All Hands Against His Own' ALLOW FILTERING"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
for r in rows:
    print(r)
                    

Row(firstname='Sara', lastname='Johnson')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Jacqueline', lastname='Lynch')


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions
table_list = session.execute("""SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'sparkify';""")
for t in table_list:
    print(t.table_name)
    session.execute(f"DROP TABLE IF EXISTS {t.table_name}")

music_library
user_log
user_table


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()